In [19]:
import cv2 as cv
import time
import os
import mediapipe as mp
import numpy as np

In [20]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [21]:
def mediapipe_detection(img, model):
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    img.flags.writeable = False
    result = model.process(img)
    img.flags.writeable = True
    img = cv.cvtColor(img, cv.COLOR_RGB2BGR)
    return img, result

In [22]:
def style_draw(img, result):
    mp_drawing.draw_landmarks(img, result.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                             mp_drawing.DrawingSpec(color = (80, 110, 10), thickness = 1, circle_radius = 1),
                             mp_drawing.DrawingSpec(color = (80, 256, 121), thickness = 1, circle_radius = 1))

    mp_drawing.draw_landmarks(img, result.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color = (80, 110, 10), thickness = 2, circle_radius = 4),
                             mp_drawing.DrawingSpec(color = (80, 44, 121), thickness = 2, circle_radius = 4))

    mp_drawing.draw_landmarks(img, result.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color = (121, 22, 76), thickness = 2, circle_radius = 4),
                             mp_drawing.DrawingSpec(color = (121, 44, 250), thickness = 2, circle_radius = 4))

    mp_drawing.draw_landmarks(img, result.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color = (245, 117, 66), thickness = 2, circle_radius = 4),
                             mp_drawing.DrawingSpec(color = (245, 66, 230), thickness = 2, circle_radius = 4))

In [23]:
def extrac_keypoints(result):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in result.pose_landmarks.landmark]).flatten() if result.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in result.face_landmarks.landmark]).flatten() if result.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in result.left_hand_landmarks.landmark]).flatten() if result.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in result.right_hand_landmarks.landmark]).flatten() if result.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [24]:
# set path for export data
Path = os.path.join('Dataset')

# set actions to detect in sign language
actions = np.array(['d'])

# number of data
no_squence = 100

# number of frame
squence_length = 60

In [25]:
for action in actions:
    for squence in range(no_squence):
        try:
            os.makedirs(os.path.join(Path, 'keypoints', action, str(squence)))
            os.makedirs(os.path.join(Path, 'video', action, str(squence)))
        except:
            pass

In [26]:
import cv2
import os

cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.8, min_tracking_confidence=0.8) as holistic:
    for action in actions:
        for sequence in range(no_squence):
            # Tentukan path untuk menyimpan video dan folder keypoints
            video_path = os.path.join(Path, 'video', action, f'{sequence}.avi')
            keypoints_folder = os.path.join(Path, 'keypoints', action, str(sequence))
            os.makedirs(keypoints_folder, exist_ok=True)
            
            # Inisialisasi objek VideoWriter untuk menyimpan video
            fourcc = cv2.VideoWriter_fourcc(*'XVID')
            vid_writer = cv2.VideoWriter(video_path, fourcc, 30, (640, 480))

            for num_frame in range(squence_length):
                ret, frame = cap.read()

                img, result = mediapipe_detection(frame, holistic)
                style_draw(img, result)

                if num_frame == 0:
                    cv.putText(img, 'STARTING FOR COLLECT', (120, 200),
                               cv.FONT_HERSHEY_DUPLEX, 1, (125, 246, 55), 4, cv.LINE_AA)
                    cv.putText(img, 'Collecting frames for {} video number {}'.format(action, sequence), (15, 12),
                               cv.FONT_HERSHEY_DUPLEX, 0.5, (125, 246, 55), 1, cv.LINE_AA)
                    cv.imshow("Collect Data", img)
                    cv.waitKey(2000)
                else:
                    cv.putText(img, 'Collecting frames for {} video number {}'.format(action, sequence), (15, 12),
                               cv.FONT_HERSHEY_DUPLEX, 0.5, (125, 246, 55), 1, cv.LINE_AA)
                    cv.imshow("Collect Data", img)
                    
                keypoints = extrac_keypoints(result)
                npy_path = os.path.join(keypoints_folder, f'{num_frame}.npy')
                np.save(npy_path, keypoints)

                # Tambahkan frame ke video
                vid_writer.write(frame)

                if cv.waitKey(10) & 0xFF == ord('q'):
                    break
            
            # Tutup objek VideoWriter setelah loop selesai
            vid_writer.release()
    
cap.release()
cv.destroyAllWindows()


In [27]:
cap.release()
cv.destroyAllWindows()